# 与信判定 予測&生成AI デモ

In [1]:
import requests
import json

import pandas as pd
import numpy as np

from h2ogpte import H2OGPTE

In [2]:
MLOPS_ENDPOINT = 'https://model.internal.dedicated.h2o.ai/dd9d2458-de72-465b-a62f-e8796053b802/model/score'

H2OGPTE_URL = 'https://playground.h2ogpte.h2o.ai'

with open('../_secret') as f:
    key = json.load(f)
H2OGPTE_KEY = key['client-access-test']

## Scoring by Predictive AI

参考: https://github.com/yukismd/H2O_AI_Cloud/blob/main/sample_request/Shaply_Scoring.ipynb

In [3]:
!ls data

UCI_Credit_Card3_sample5.csv


In [20]:
def get_and_transform_scoringdata() -> dict:
    df = pd.read_csv('data/UCI_Credit_Card3_sample5.csv')
    df = df.sample(n=1)   # sample 1 observation out of 5
    df = df.astype(str)
    display(df)
    
    # jsonに変換
    json_data = df.to_json(orient="split")
    json_data = json.loads(json_data)
    # キー名の変更
    json_data['fields'] = json_data['columns']
    del json_data['columns']
    json_data['rows'] = json_data['data']
    del json_data['data']
    # Shapley Scoring指示の追加
    json_data['requestShapleyValueType'] = 'ORIGINAL'
    
    return json_data

In [21]:
scoring_data = get_and_transform_scoringdata()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default_payment_next_month
0,1,20000,M,university,married,24,2,2,-1,-1,...,0,0,0,0,689,0,0,0,0,1


In [22]:
scoring_data

{'index': [0],
 'fields': ['ID',
  'LIMIT_BAL',
  'SEX',
  'EDUCATION',
  'MARRIAGE',
  'AGE',
  'PAY_1',
  'PAY_2',
  'PAY_3',
  'PAY_4',
  'PAY_5',
  'PAY_6',
  'BILL_AMT1',
  'BILL_AMT2',
  'BILL_AMT3',
  'BILL_AMT4',
  'BILL_AMT5',
  'BILL_AMT6',
  'PAY_AMT1',
  'PAY_AMT2',
  'PAY_AMT3',
  'PAY_AMT4',
  'PAY_AMT5',
  'PAY_AMT6',
  'default_payment_next_month'],
 'rows': [['1',
   '20000',
   'M',
   'university',
   'married',
   '24',
   '2',
   '2',
   '-1',
   '-1',
   '-2',
   '-2',
   '3913',
   '3102',
   '689',
   '0',
   '0',
   '0',
   '0',
   '689',
   '0',
   '0',
   '0',
   '0',
   '1']],
 'requestShapleyValueType': 'ORIGINAL'}

In [23]:
response = requests.post(url=MLOPS_ENDPOINT, 
                         headers={'content-type': 'application/json'}, 
                         data=json.dumps(scoring_data))

In [24]:
type(response)

requests.models.Response

In [25]:
response.json()['fields'][1], response.json()['score'][0][1]

('default_payment_next_month.1', '0.6974739273081736')

In [26]:
def scoring_results(response: requests.models.Response) -> (float, pd.DataFrame):
    # Scoring result
    pred_prob = float(response.json()['score'][0][1])   # probability of y=1
    
    # Shaply scoring result
    df_contrib = pd.DataFrame({
        'features':response.json()['featureShapleyContributions']['features'],
        'contributions':response.json()['featureShapleyContributions']['contributionGroups'][0]['contributions'][0],
    })
    df_contrib['contributions'] = df_contrib['contributions'].astype(np.float64)
    df_contrib['features'] = [s.replace('contrib_', '') for s in df_contrib['features']]
    
    return (pred_prob, df_contrib)

In [27]:
pred_prob, df_contrib = scoring_results(response)
print(pred_prob)
display(df_contrib)

0.6974739273081736


,features,contributions
0,EDUCATION,0.096083
1,MARRIAGE,0.087222
2,AGE,0.080195
3,PAY_1,0.794107
4,PAY_2,0.323085
5,PAY_3,0.002936
6,PAY_4,0.059302
7,PAY_5,0.039223
8,PAY_6,-0.010183
9,BILL_AMT1,0.045825


In [45]:
shap = 1
1 / (1 + np.exp(-1*shap))

0.7310585786300049

## Interpretation by Generative AI

参考: https://github.com/yukismd/H2O_LLM/blob/main/h2oGPTe/Collection-specific-API.ipynb

In [28]:
client = H2OGPTE(
    address=H2OGPTE_URL,
    api_key=H2OGPTE_KEY,
)

client

In [30]:
client.get_llm_names()

['mistralai/Mixtral-8x7B-Instruct-v0.1',
 'meta-llama/Meta-Llama-3-8B-Instruct',
 'meta-llama/Meta-Llama-3-70B-Instruct',
 'nvidia/Llama3-ChatQA-1.5-70B',
 'NousResearch/Nous-Capybara-34B',
 'mistralai/Mistral-7B-Instruct-v0.3',
 'h2oai/h2o-danube2-1.8b-chat',
 'mistralai/Mistral-Nemo-Instruct-2407',
 'OpenGVLab/InternVL-Chat-V1-5',
 'THUDM/cogvlm2-llama3-chat-19B',
 'liuhaotian/llava-v1.6-34b',
 'lmms-lab/llama3-llava-next-8b',
 'mistral-small-latest',
 'mistral-large-latest',
 'mistral-medium',
 'claude-3-5-sonnet-20240620',
 'claude-3-sonnet-20240229',
 'claude-3-opus-20240229',
 'claude-3-haiku-20240307',
 'microsoft/Phi-3-vision-128k-instruct',
 'microsoft/Phi-3-medium-128k-instruct',
 'google/gemma-2-27b-it',
 'gemini-1.5-pro-latest',
 'gemini-1.5-flash-latest',
 'gpt-3.5-turbo-0613',
 'gpt-3.5-turbo-16k-0613',
 'gpt-35-turbo-1106',
 'gpt-4-1106-preview',
 'gpt-4-turbo-2024-04-09']

In [29]:
chat_session_id = client.create_chat_session_on_default_collection()
chat_session_id

'1e552bde-0169-40c9-99ca-846fe1c4582d'

In [32]:
print(pred_prob)
#display(df_contrib)

0.6974739273081736


In [55]:
SysPrompt = '''
あなたは優秀な与信判定AIです。（回答は必ず日本語で実施します。）
'''

Message = '''
予測モデルの予測結果: {}
予測モデルの予測結果を、以下の与信判断の目安の目安に従って、与信可否の判定をして下さい。

### 与信判断の目安
予測精度が0.3以下: 与信判定に問題なし。貸し出し可能。
予測精度が0.3~0.7: 与信判定に問題ありの可能性。リーズンコードの確認が必要。
予測精度が0.7以上: 与信判定に問題あり。貸し出し不可。

'''.format(pred_prob)

LlmModel = 'microsoft/Phi-3-medium-128k-instruct'

with client.connect(chat_session_id) as session:
    reply = session.query(
        message = Message,
        system_prompt=SysPrompt,
        llm=LlmModel,
        rag_config={"rag_type": "llm_only",},
    )
    print(reply.content)

与信判定の目安に従って、与信可否の判定を行います。

与信モデルの予測結果は0.6974739273081736です。この値は0.3から0.7の範囲にあります。したがって、与信判定に問題がある可能性があり、リーズンコードの確認が必要です。


In [48]:
df_contrib[df_contrib['contributions']>0]

,features,contributions
0,EDUCATION,0.096083
1,MARRIAGE,0.087222
2,AGE,0.080195
3,PAY_1,0.794107
4,PAY_2,0.323085
5,PAY_3,0.002936
6,PAY_4,0.059302
7,PAY_5,0.039223
9,BILL_AMT1,0.045825
10,BILL_AMT2,0.269943


In [60]:
list(df_contrib[df_contrib['contributions']>0]['features'])

['EDUCATION',
 'MARRIAGE',
 'AGE',
 'PAY_1',
 'PAY_2',
 'PAY_3',
 'PAY_4',
 'PAY_5',
 'BILL_AMT1',
 'BILL_AMT2',
 'PAY_AMT1',
 'PAY_AMT2',
 'PAY_AMT3',
 'PAY_AMT4',
 'PAY_AMT5',
 'PAY_AMT6']

In [62]:
SysPrompt = '''
あなたは優秀な与信判定AIです。（回答は必ず日本語で実施します。）
'''

Message = '''
予測モデルの予測結果: {}
予測モデルの予測結果を、以下の与信判断の目安の目安に従って、与信可否の判定をして下さい。

リーズンコードの結果:{}
与信可否の判定において、リーズンコードの確認が必要な場合は、リーズンコードの結果に従い、以下のリーズンコードの基準に理由を述べて下さい。

### 与信判断の目安
予測精度が0.3以下: 与信判定に問題なし。貸し出し可能。
予測精度が0.3~0.7: 与信判定に問題ありの可能性。リーズンコードの確認が必要。
予測精度が0.7以上: 与信判定に問題あり。貸し出し不可。

### リーズンコードの基準
PAY_*, BILL_AMT*, PAY_AMT*が8個以上含まれる場合は、過去の返済履歴に問題ありとなります。
PAY_*, BILL_AMT*, PAY_AMT*が8個未満であれば含まれる場合は、返済履歴には問題がありません。

'''.format(pred_prob, list(df_contrib[df_contrib['contributions']>0]['features']))

LlmModel = 'microsoft/Phi-3-medium-128k-instruct'

with client.connect(chat_session_id) as session:
    reply = session.query(
        message = Message,
        system_prompt=SysPrompt,
        llm=LlmModel,
        rag_config={"rag_type": "llm_only",},
    )
    print(reply.content)

与信判定の目安に従って、与信可否の判定を行います。

与信モデルの予測結果は0.6974739273081736で、これは予測精度が0.3~0.7の範囲にあります。したがって、与信判定に問題ありの可能性があり、リーズンコードの確認が必要です。

リーズンコードの結果は以下の通りです:
['EDUCATION', 'MARRIAGE', 'AGE', 'PAY_1', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'BILL_AMT1', 'BILL_AMT2', 'PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6']

リーズンコードの基準によると、PAY_*, BILL_AMT*, PAY_AMT*が8個以上含まれています。これは過去の返済履歴に問題があることを示しています。

以上の情報に基づき、与信判定に問題ありの可能性があり、貸し出し不可と判断します。


In [64]:
# LLMに投げたプロンプト
print(Message)


予測モデルの予測結果: 0.6974739273081736
予測モデルの予測結果を、以下の与信判断の目安の目安に従って、与信可否の判定をして下さい。

リーズンコードの結果:['EDUCATION', 'MARRIAGE', 'AGE', 'PAY_1', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'BILL_AMT1', 'BILL_AMT2', 'PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6']
与信可否の判定において、リーズンコードの確認が必要な場合は、リーズンコードの結果に従い、以下のリーズンコードの基準に理由を述べて下さい。

### 与信判断の目安
予測精度が0.3以下: 与信判定に問題なし。貸し出し可能。
予測精度が0.3~0.7: 与信判定に問題ありの可能性。リーズンコードの確認が必要。
予測精度が0.7以上: 与信判定に問題あり。貸し出し不可。

### リーズンコードの基準
PAY_*, BILL_AMT*, PAY_AMT*が8個以上含まれる場合は、過去の返済履歴に問題ありとなります。
PAY_*, BILL_AMT*, PAY_AMT*が8個未満であれば含まれる場合は、返済履歴には問題がありません。


